In [ ]:
import jax
from jax import numpy as jnp

In [ ]:
H = 6
action_dim = 2
N = 1000
key = jax.random.PRNGKey(0)

In [ ]:
a_mean = jnp.linspace(0, 1, action_dim*H).reshape((H, action_dim))
a_cov = jnp.tile(jnp.eye(action_dim)*0.04, (H, 1, 1))

sample_key, key = jax.random.split(key)
keys = jax.random.split(sample_key, N)
def single_sample(key, traj_mean, traj_cov):
    return jax.vmap(lambda mean, cov: jax.random.multivariate_normal(key, mean, cov))(traj_mean, traj_cov)
# repeat single_sample N times to get N samples
a_sampled = jax.vmap(single_sample, in_axes=(0, None, None))(keys, a_mean, a_cov)
a_sampled_flattened = jnp.reshape(a_sampled, (N, action_dim*H))

In [ ]:
def single_sample(key):
    return jax.random.multivariate_normal(key, a_mean.flatten(), jnp.eye(2*H)*0.04)
sample_key, key = jax.random.split(key)
keys = jax.random.split(sample_key, N)
a_sampled_flattened_new = jax.vmap(single_sample)(keys)

In [ ]:
# get covariance matrix
a_old_cov = jnp.cov(a_sampled_flattened.T)
a_old_cov.shape

In [ ]:
# import pickle

# with open('/home/pcy/Research/quadjax/results/a_sampled_old.pkl', 'rb') as f:
#     a_sampled_old = pickle.load(f)
# with open('/home/pcy/Research/quadjax/results/a_sampled_new.pkl', 'rb') as f:
#     a_sampled_new = pickle.load(f)
# N = 128
# H = 32
# action_dim = 2
# a_sampled_flattened = a_sampled_old.reshape(128, -1)
# a_sampled_flattened_new = a_sampled_new.reshape(128, -1)

In [ ]:
# plot a_sampled_flattened distribution with bars
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()
for i in range(action_dim*H):
    plt.figure()
    plt.hist(a_sampled_flattened[:, i], bins=50, alpha=0.5, label='old')
    plt.hist(a_sampled_flattened_new[:, i], bins=50, alpha=0.5, label='new')
    plt.legend()

In [1]:
import numpy as np

p = np.array([1.71479058e-09,  8.80284482e-05, -2.21152097e-01])

def pwm_to_force(pwm):
    force_in_grams = np.polyval(p, pwm)
    force_in_newton = force_in_grams * 9.81 / 1000.0
    return np.maximum(force_in_newton, 0)

def force_to_pwm(force):
    a, b, c = p
    force_in_grams = force * 1000.0 / 9.81
    pwm = (-b + np.sqrt(b**2 - 4*a*(c-force_in_grams))) / (2*a)
    return pwm

force_to_pwm(pwm_to_force(65535))

65535.000000000015

In [3]:
arm = 0.046 * 0.707
max_force = 0.5067/4
2 * max_force * arm

0.008239448699999999